In [51]:
import pandas as pd
from sklearn.metrics import accuracy_score
from sklearn.impute import SimpleImputer
import joblib

In [52]:
file_path = 'Asset50k.xlsx'
df = pd.read_excel(file_path)

In [53]:
X = df.iloc[:, 1:]  # Independent variables
y = df['AssetType']  # Dependent variable

In [54]:
X.head()

,SerialNo,Manufacturer,ModelNbr
0,NaN,NaN,NaN
1,NaN,NaN,NaN
2,NaN,NaN,NaN
3,0,NaN,O
4,NaN,NaN,FRENGER


In [55]:
special_values = [1234, 'UNKOWN', 'Unknown', 'NA', 'NULL', 'UNKNOWN', 'TBA', 'N/A', 'NOT VISIBLE', '123TEST', 'UNABLE TO LOCATE', 'NO ID', 'NO ACCESS', 'UNKOWN', 'NaN', 'na', 'AS PER PICS']

# Create a copy of the DataFrame to avoid SettingWithCopyWarning
X_copy = X.copy()

# Replace values using .loc to avoid SettingWithCopyWarning
X_copy.loc[:, 'SerialNo'] = X_copy['SerialNo'].replace(special_values, pd.NA)
X_copy.loc[:, 'Manufacturer'] = X_copy['Manufacturer'].replace(special_values, pd.NA)
X_copy.loc[:, 'ModelNbr'] = X_copy['ModelNbr'].replace(special_values, pd.NA)

X = X_copy

In [56]:
label_encoder = joblib.load('model/label_encoder.joblib')

In [57]:
X['ModelNbr'] = label_encoder.fit_transform(X['ModelNbr'].astype(str))
X['SerialNo'] = label_encoder.fit_transform(X['SerialNo'].astype(str))
X['Manufacturer'] = label_encoder.fit_transform(X['Manufacturer'].astype(str))

y = label_encoder.fit_transform(y.astype(str))

In [44]:
# Impute missing values
imputer = SimpleImputer(strategy='most_frequent')
X_imputed = pd.DataFrame(imputer.fit_transform(X), columns=X.columns)

C:\ML\lib\site-packages\sklearn\impute\_base.py:49: FutureWarning: Unlike other reduction functions (e.g. `skew`, `kurtosis`), the default behavior of `mode` typically preserves the axis it acts along. In SciPy 1.11.0, this behavior will change: the default value of `keepdims` will become False, the `axis` over which the statistic is taken will be eliminated, and the value None will no longer be accepted. Set `keepdims` to True or False to avoid this warning.
  mode = stats.mode(array)


In [58]:
rf_model = joblib.load('model/rf_60')

C:\ML\lib\site-packages\sklearn\base.py:329: UserWarning: Trying to unpickle estimator DecisionTreeClassifier from version 1.2.2 when using version 1.0.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/modules/model_persistence.html#security-maintainability-limitations
  warnings.warn(
C:\ML\lib\site-packages\sklearn\base.py:329: UserWarning: Trying to unpickle estimator RandomForestClassifier from version 1.2.2 when using version 1.0.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/modules/model_persistence.html#security-maintainability-limitations
  warnings.warn(


In [61]:
chunk_size = 20000
X_test_chunk, _, y_test_chunk, _ = train_test_split(X_imputed, y, test_size=chunk_size, random_state=42)
y_pred_encoded = rf_model.predict(X_test_chunk)

In [62]:
accuracy = accuracy_score(y_test_chunk, y_pred_encoded)
print(f"Accuracy on the chunk: {accuracy}")

Accuracy on the chunk: 0.5918536496526706


In [63]:
y_pred_encoded

array([150, 398, 285, ..., 533, 540, 526], dtype=int64)

In [64]:
y_pred_actual = label_encoder.inverse_transform(y_pred_encoded)

In [65]:
y_pred_actual

array(['CASSETTE SPLIT SYSTEM', 'MULTI HEAD DUCTED SPLIT SYSTEM',
       'FIRE PANEL-GRAPHICS SYSTEM', ..., 'VRV CASSETTE SPLIT',
       'WALL MOUNTED SPLIT SYSTEM', 'VARIABLE SPEED DRIVE'], dtype=object)